<a href="https://colab.research.google.com/github/HeptaDecane/LP1_SEM7/blob/main/C02/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://raw.githubusercontent.com/HeptaDecane/LP1_SEM7/main/C02/diabetes.csv --output diabetes.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23873  100 23873    0     0   158k      0 --:--:-- --:--:-- --:--:--  158k


In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
column_map = {
    'Pregnancies': 'pregnancies',
    'Glucose': 'glucose',
    'BloodPressure': 'blood_pressure',
    'SkinThickness': 'skin_thickness',
    'Insulin': 'insulin',
    'BMI': 'bmi',
    'DiabetesPedigreeFunction': 'pedigree_function',
    'Age': 'age',
    'Outcome': 'outcome'
}
df.rename(columns=column_map,inplace=True)
df.head()

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,pedigree_function,age,outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pregnancies        768 non-null    int64  
 1   glucose            768 non-null    int64  
 2   blood_pressure     768 non-null    int64  
 3   skin_thickness     768 non-null    int64  
 4   insulin            768 non-null    int64  
 5   bmi                768 non-null    float64
 6   pedigree_function  768 non-null    float64
 7   age                768 non-null    int64  
 8   outcome            768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
df['outcome'].value_counts()

0    500
1    268
Name: outcome, dtype: int64

In [7]:
x = df.drop(['outcome'],axis=1)
y = df['outcome']

In [8]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=73)

In [9]:
x_train.describe()

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,pedigree_function,age
count,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000
mean,3.859922,122.120623,69.684825,20.982490,80.513619,32.228988,0.468996,33.536965
std,3.418763,33.464494,19.008100,15.941342,119.020093,8.001094,0.306388,12.097863
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,100.000000,64.000000,0.000000,0.000000,27.400000,0.251250,24.000000
50%,3.000000,119.000000,72.000000,23.000000,17.000000,32.400000,0.377000,29.000000
75%,6.000000,142.000000,80.000000,32.000000,129.750000,36.775000,0.622750,41.000000
max,17.000000,198.000000,114.000000,99.000000,846.000000,67.100000,2.329000,81.000000


In [10]:
train_mean_pos = x_train[y_train==1].mean()
train_std_pos = x_train[y_train==1].std()
train_mean_neg = x_train[y_train==0].mean()
train_std_neg = x_train[y_train==0].std()

In [11]:
from math import sqrt
from math import pi
from math import exp

def cond_probability(x, mean, std):
    exponent = exp(-((x - mean)**2/(2*std**2)))
    return (1 / (sqrt(2*pi)*std)) * exponent

In [12]:
def predict(row):
    prob_pos = len(x_train[y_train==1]) / len(x_train)
    
    for i in range(0,len(row)):
        prob_pos = prob_pos * cond_probability(row[i],train_mean_pos[i],train_std_pos[i])
        
    prob_neg = len(x_train[y_train==0]) / len(x_train)
    
    for i in range(0,len(row)):
        prob_neg = prob_neg * cond_probability(row[i],train_mean_neg[i],train_std_neg[i])    
    
    return [prob_pos,prob_neg]

In [13]:
predictions_raw = []

for row in x_test.values.tolist():
    predictions_raw.append(predict(row))

In [14]:
predictions_raw[0]

[7.034504006997928e-14, 1.5354754946719242e-12]

In [15]:
predictions = []
for row in predictions_raw:
    if(row[0]>row[1]):
        predictions.append(1)
    else:
        predictions.append(0)

In [16]:
metrics.accuracy_score(y_test.tolist(),predictions)

0.7480314960629921

In [17]:
matrix = metrics.confusion_matrix(y_test.tolist(),predictions)
matrix_df = pd.DataFrame(data=matrix,index=['-ve','+ve'],columns=['predicted -ve','predicted +ve'])
matrix_df

,predicted -ve,predicted +ve
-ve,147,27
+ve,37,43


In [18]:
from sklearn.naive_bayes import GaussianNB

In [19]:
model = GaussianNB()
model.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [20]:
model.score(x_test,y_test)

0.7480314960629921

In [21]:
test_set_prediction = model.predict(x_test)
matrix = metrics.confusion_matrix(y_test,test_set_prediction)
matrix_df = pd.DataFrame(data=matrix,index=['-ve','+ve'],columns=['predicted -ve','predicted +ve'])
matrix_df

,predicted -ve,predicted +ve
-ve,147,27
+ve,37,43
